In [1]:
number = '7.5.1'

In [2]:
CONFIG = {
    'data_folder' : 'C:/Users/Николай/PycharmProjects/VKRecSys/data/',
    'val_pred_folder' : 'C:/Users/Николай/PycharmProjects/VKRecSys/C.Results/val/',
    'test_pred_folder' : 'C:/Users/Николай/PycharmProjects/VKRecSys/C.Results/test/',
    
    'train_path' : 'train_interactions.parquet',
    'test_path': 'test_pairs.csv',  
    'items_meta_path' : 'items_meta.parquet',
    'users_meta_path' : 'users_meta.parquet',
    'folds_path' : 'fold.csv',
    'val_output_path' : f'{number}_val',
    'test_output_path' : f'{number}_test',
    
    'user_emb_size' : 256, 
    'item_emb_size' : 256, 
    'source_emb_size' : 256, 
    'age_emb_size' : 256, 
    'duration_emb_size' : 256, 
    'gender_emb_size' : 256, 
    
    'DEVICE' : 'cuda',
    'SEED' : 42,
    'BATCH_SIZE' : 16384,
    'LR' : 0.001,
    'EPOCHS' : 5,
    'output_dim' : 3,
    
    'share_weight': 0.7332961900532841,
    'bookmark_weight': 0.05949911244473274,
    'timespent_weight': 0.09469452073341826
    
}

In [3]:
# Import libs
import numpy as np
from tqdm import tqdm
import pandas as pd
import torch.nn.functional as F
import torch
import torch.nn as nn
from torch.optim import Adam

In [4]:
# Device, torch decimal places and seed for reproducibility
device = torch.device(CONFIG['DEVICE'] if torch.cuda.is_available() else "cpu")
torch.set_printoptions(precision=40) 

torch.manual_seed(CONFIG['SEED'])  
torch.cuda.manual_seed_all(CONFIG['SEED'])  
np.random.seed(CONFIG['SEED'])  

In [5]:
# Load and prepare data
train = pd.read_parquet(f"{CONFIG['data_folder']}{CONFIG['train_path']}", engine='pyarrow')
train['target'] = train['like'] + train['dislike'].replace({1: -1})
train.drop(columns=['like', 'dislike'], inplace=True)
train['target'] = train['target'].astype('int8')
train['target'] = train['target'].replace({-1:0, 0:1, 1:2})
train['timespent'] = train['timespent'] - 1
train = train.iloc[:100000]

test = pd.read_csv(f"{CONFIG['data_folder']}{CONFIG['test_path']}")
test_to_save = test.copy()

items_meta = pd.read_parquet(f"{CONFIG['data_folder']}{CONFIG['items_meta_path']}", engine='pyarrow')
items_meta['duration'] = items_meta['duration'] - 5
items_meta['item_id'] = items_meta['item_id'].astype('category')
items_meta['source_id'] = items_meta['source_id'].astype('category')
items_meta.set_index('item_id', inplace=True)

users_meta = pd.read_parquet(f"{CONFIG['data_folder']}{CONFIG['users_meta_path']}", engine='pyarrow')
users_meta['age'] = users_meta['age'] - 18
users_meta['gender'] = users_meta['gender'].replace({1:0, 2:1})
users_meta['user_id'] = users_meta['user_id'].astype('category')
users_meta['gender'] = users_meta['gender'].astype('category')
users_meta.set_index('user_id', inplace=True)

In [6]:
# Model definition
class Model(nn.Module):
    def __init__(self, 
                 input_dim, 
                 num_users=users_meta.index.nunique(), 
                 num_items=items_meta.index.nunique(), 
                 num_sources=items_meta['source_id'].nunique(),
                 num_ages=users_meta['age'].nunique(),
                 num_durations=items_meta['duration'].nunique(),
                 num_genders=users_meta['gender'].nunique(), 
                 output_dim=CONFIG['output_dim']): 
        
        super(Model, self).__init__()
        self.user_embedding = nn.Embedding(num_users, CONFIG['user_emb_size'])
        self.item_embedding = nn.Embedding(num_items, CONFIG['item_emb_size'])
        self.source_embedding = nn.Embedding(num_sources, CONFIG['source_emb_size'])
        self.age_embedding = nn.Embedding(num_ages, CONFIG['age_emb_size'])
        self.duration_embedding = nn.Embedding(num_durations, CONFIG['duration_emb_size'])
        self.gender_embedding = nn.Embedding(num_genders, CONFIG['gender_emb_size'])
         
        self.fc1 = nn.Linear(input_dim, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 256)
        self.fc4 = nn.Linear(256, 128)
        self.fc5 = nn.Linear(128, 64)
        self.fc6 = nn.Linear(64, 32)
        self.fc7 = nn.Linear(32, 16)
        self.fc8 = nn.Linear(16, 8)
        self.target_output = nn.Linear(8, output_dim)
        self.share_output = nn.Linear(8, 1)
        self.bookmark_output = nn.Linear(8, 1)
        self.timespent_output = nn.Linear(8, 255)
        
        self.gelu = nn.GELU()
    
    def forward(self, user_ids, item_ids, source_ids, age_ids, duration_ids, gender_ids, embeddings):

        user_emb = self.user_embedding(user_ids)
        item_emb = self.item_embedding(item_ids)
        source_emb = self.source_embedding(source_ids)
        age_emb = self.age_embedding(age_ids)
        duration_emb = self.duration_embedding(duration_ids)
        gender_emb = self.gender_embedding(gender_ids)
        
        x = torch.cat((user_emb, item_emb, source_emb, age_emb, duration_emb, gender_emb, embeddings), dim=1)
        
        x = self.gelu(self.fc1(x))
        x = self.gelu(self.fc2(x))
        x = self.gelu(self.fc3(x))
        x = self.gelu(self.fc4(x))
        x = self.gelu(self.fc5(x))
        x = self.gelu(self.fc6(x))
        x = self.gelu(self.fc7(x))
        x = self.gelu(self.fc8(x))
        target_output = self.target_output(x)
        share_output = self.share_output(x)
        bookmark_output = self.bookmark_output(x)
        timespent_output = self.timespent_output(x)
        
        return target_output, share_output, bookmark_output, timespent_output

In [7]:
# Input dimension
input_dim = (CONFIG['user_emb_size'] + 
             CONFIG['item_emb_size'] + 
             CONFIG['source_emb_size'] + 
             CONFIG['age_emb_size'] +
             CONFIG['duration_emb_size'] + 
             CONFIG['gender_emb_size'] + 
             32)

In [8]:
# Model, criterion and optimizer
model = Model(input_dim).to(device)
target_criterion = nn.CrossEntropyLoss()
share_criterion = nn.BCEWithLogitsLoss()
bookmark_criterion = nn.BCEWithLogitsLoss()
timespent_criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=CONFIG['LR'])

In [ ]:
# Training
train_num_samples = len(train)
train_num_batches = (train_num_samples + CONFIG['BATCH_SIZE'] - 1) // CONFIG['BATCH_SIZE']

test_num_samples = len(test)
test_num_batches = (test_num_samples + CONFIG['BATCH_SIZE'] - 1) // CONFIG['BATCH_SIZE']

for epoch in range(CONFIG['EPOCHS']):
##################################################################TRAIN##################################################################
    model.train()
    train_total_running_loss = 0.0
    train_target_running_loss = 0.0 
    with tqdm(range(train_num_batches), desc=f"Epoch {epoch + 1}/{CONFIG['EPOCHS']}", unit="batch") as t:
        for batch_idx in t:
            start_idx = batch_idx * CONFIG['BATCH_SIZE']
            end_idx = min(start_idx + CONFIG['BATCH_SIZE'], train_num_samples)
            
            batch_main = train.iloc[start_idx:end_idx]
            
            batch_user_values = batch_main['user_id'].values
            batch_item_values = batch_main['item_id'].values

            batch_users_meta = users_meta.loc[batch_user_values]
            batch_items_meta = items_meta.loc[batch_item_values]

            batch_user_values = torch.tensor(batch_user_values, dtype=torch.long, device=device)
            batch_item_values = torch.tensor(batch_item_values, dtype=torch.long, device=device)

            batch_gender_values = torch.tensor(batch_users_meta['gender'].values, dtype=torch.long, device=device)
            batch_age_values = torch.tensor(batch_users_meta['age'].values, dtype=torch.long, device=device)
            batch_source_values = torch.tensor(batch_items_meta['source_id'].values, dtype=torch.long, device=device)
            batch_duration_values = torch.tensor(batch_items_meta['duration'].values, dtype=torch.long, device=device)

            embeddings = torch.tensor(np.stack(batch_items_meta['embeddings'].values), device=device, dtype=torch.float32)
            targets = torch.tensor(batch_main['target'].values, dtype=torch.long, device=device)
            shares = torch.tensor(batch_main['share'].values, dtype=torch.float32, device=device).unsqueeze(1)
            bookmarks = torch.tensor(batch_main['bookmarks'].values, dtype=torch.float32, device=device).unsqueeze(1)
            timespents = torch.tensor(batch_main['timespent'].values, dtype=torch.long, device=device)

            optimizer.zero_grad()
            
            target_outputs, share_outputs, bookmark_outputs, timespent_outputs = model(batch_user_values,
                                                                                           batch_item_values, 
                                                                                           batch_source_values,
                                                                                           batch_age_values, 
                                                                                           batch_duration_values, 
                                                                                           batch_gender_values, 
                                                                                           embeddings)
            train_target_loss = target_criterion(target_outputs, targets)
            train_target_running_loss += train_target_loss.item()
            train_share_loss = share_criterion(share_outputs, shares) * CONFIG['share_weight']
            train_bookmark_loss = bookmark_criterion(bookmark_outputs, bookmarks) * CONFIG['bookmark_weight']
            train_timespent_loss = timespent_criterion(timespent_outputs, timespents) * CONFIG['timespent_weight']

            train_total_loss = train_target_loss + train_share_loss + train_bookmark_loss + train_timespent_loss
            train_total_loss.backward()
            optimizer.step()

            train_total_running_loss += train_total_loss.item()
            t.set_postfix(train_loss=f"{train_total_running_loss / (batch_idx + 1):.6f}",
                          target_loss=f"{train_target_running_loss / (batch_idx + 1):.6f}")
        
##################################################################EVAL##################################################################
    model.eval()
    test_running_loss = 0.0
    
    outputs_list = []

    with torch.no_grad():
        with tqdm(range(test_num_batches), desc=f"Epoch {epoch + 1}/{CONFIG['EPOCHS']}", unit="batch") as v:
            for batch_idx in v:
                start_idx = batch_idx * CONFIG['BATCH_SIZE']
                end_idx = min(start_idx + CONFIG['BATCH_SIZE'], test_num_samples)
                
                batch_main = test.iloc[start_idx:end_idx]
                
                batch_user_values = batch_main['user_id'].values
                batch_item_values = batch_main['item_id'].values
    
                batch_users_meta = users_meta.loc[batch_user_values]
                batch_items_meta = items_meta.loc[batch_item_values]
    
                batch_user_values = torch.tensor(batch_user_values, dtype=torch.long, device=device)
                batch_item_values = torch.tensor(batch_item_values, dtype=torch.long, device=device)
    
                batch_gender_values = torch.tensor(batch_users_meta['gender'].values, dtype=torch.long, device=device)
                batch_age_values = torch.tensor(batch_users_meta['age'].values, dtype=torch.long, device=device)
                batch_source_values = torch.tensor(batch_items_meta['source_id'].values, dtype=torch.long, device=device)
                batch_duration_values = torch.tensor(batch_items_meta['duration'].values, dtype=torch.long, device=device)
    
                embeddings = torch.tensor(np.stack(batch_items_meta['embeddings'].values), device=device, dtype=torch.float32)

                target_outputs, share_outputs, bookmark_outputs, timespent_outputs = model(batch_user_values,
                                                                                           batch_item_values, 
                                                                                           batch_source_values,
                                                                                           batch_age_values, 
                                                                                           batch_duration_values, 
                                                                                           batch_gender_values, 
                                                                                           embeddings)
               
                
                probabilities = F.softmax(target_outputs, dim=1)
                class_weights = torch.tensor([0, 1, 2], device=probabilities.device, dtype=probabilities.dtype)
                weighted_predictions = torch.sum(probabilities * class_weights, dim=1).cpu().numpy()
        
                outputs_list.extend(weighted_predictions)

##################################################################SAVE##################################################################
    df_outputs = pd.DataFrame(outputs_list, columns=['predict'])
    test_to_save['predict'] = df_outputs['predict']
    output_path = f"{CONFIG['test_pred_folder']}{CONFIG['test_output_path']}_e{epoch}.csv"
    test_to_save.to_csv(output_path, index=False)

    train_loss = train_total_running_loss / train_num_batches

    print('Outputs saved at', output_path)
    print(f"Epoch [{epoch + 1}/{CONFIG['EPOCHS']}]: Train Loss: {train_loss:.6f}")

Epoch 1/5: 100%|██████████| 102/102 [00:11<00:00,  9.08batch/s]


Outputs saved at C:/Users/Николай/PycharmProjects/VKRecSys/C.Results/test/7.5.1_test_e0.csv
Epoch [1/5]: Train Loss: 1.636366


Epoch 2/5: 100%|██████████| 102/102 [00:11<00:00,  8.70batch/s]
